In [1]:
# 5/29
# 파일이 너무 커서 머신러닝에 어려움이 있어서 10분의 1 로 샘플링해서 저장후 다시 불러와서 머신러닝 시작
import pandas as pd
import numpy as np
# 파일 저장및 불러오기 모듈
import bz2
import pickle
import _pickle as cPickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data
# 불러온 데이터는 전처리 완료된 데이터들이 분할된 형태
data = decompress_pickle('texi_train/adddate.pbz2')
data

,index,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,year,month,day,hour,distance,date
16973830,52263852,10.9,2011-01-13 11:20:00+00:00,-73.957230,40.768910,-73.977330,40.757120,2011,1,13,11,0.023303,4
50034343,43209749,10.5,2009-12-02 08:34:00+00:00,-73.982773,40.721264,-74.006119,40.705658,2009,12,2,8,0.028082,4
21507333,10798948,8.5,2010-08-19 09:59:00+00:00,-73.991150,40.755035,-73.995796,40.743389,2010,8,19,9,0.012539,5
21340781,10285876,17.0,2012-09-22 21:41:00+00:00,-73.955482,40.765823,-74.003212,40.732883,2012,9,22,21,0.057993,5
22732867,14570581,4.5,2011-07-31 21:35:00+00:00,-73.937157,40.797829,-73.948235,40.797256,2011,7,31,21,0.011093,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35781699,54747850,14.5,2010-05-08 00:31:00+00:00,-73.991417,40.685246,-73.985046,40.719112,2010,5,8,0,0.034461,7
24537187,20126125,7.5,2013-09-29 10:12:00+00:00,-73.988068,40.774788,-73.964958,40.769314,2013,9,29,10,0.023749,1
34836143,51836212,6.5,2011-01-07 15:31:00+00:00,-73.961243,40.765144,-73.975494,40.760227,2011,1,7,15,0.015076,5
40436048,13654877,13.5,2014-02-12 12:13:00+00:00,-73.789612,40.647606,-73.771423,40.659283,2014,2,12,12,0.021614,3


In [5]:
a = pd.get_dummies(data[["year",'month','hour','date']])
x_data =data[["pickup_longitude","pickup_latitude", "dropoff_longitude", "dropoff_latitude", "distance"]]
y_target = data["fare_amount"]
a[["pickup_longitude","pickup_latitude", "dropoff_longitude", "dropoff_latitude", "distance"]] = data[["pickup_longitude","pickup_latitude", "dropoff_longitude", "dropoff_latitude", "distance"]]
x_data = a

In [3]:
# x_data =data[["pickup_longitude","pickup_latitude", "dropoff_longitude", "dropoff_latitude", "year", "month", "hour", "distance"]]
# y_target = data["fare_amount"]

In [6]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_target, test_size=0.2, random_state=1)

lr_reg = LinearRegression()
lr_reg.fit(x_train, y_train)
pred = lr_reg.predict(x_test)
print(pred)

[ 7.60015869 10.34307861 10.99713135 ...  8.95758057  8.02172852
  5.83807373]


In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
mae_val = mean_absolute_error(y_test, pred)
print("MAE: {0:.3F}".format(mae_val))
mse_val = mean_squared_error(y_test, pred)
print("MSE: {0:.3F}".format(mse_val))
rmse_val = np.sqrt(mse_val)
print("RMSE: {0: .3F}".format(rmse_val))

MAE: 2.151
MSE: 19.779
RMSE:  4.447


In [9]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [10]:
#include_bias=False => 절편 계산 생략
p_model = Pipeline([ ('poly',PolynomialFeatures(degree=3, include_bias=True)), ('linear',LinearRegression()) ])
p_model.fit(x_train, y_train)
y_pred = p_model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(round(mse,3),  round(rmse,3))
print("Variance score : {0:.3f}".format(r2_score(y_test, y_pred)))

MemoryError: Unable to allocate 993. GiB for an array with shape (4320000, 30856) and data type float64

In [11]:
degrees = [1, 2, 4, 5, 8]

for i in range(len(degrees)):   
    
    polynomial_features = PolynomialFeatures(degree = degrees[i], include_bias=False)
    lr =LinearRegression()
    pipeline = Pipeline([("poly",polynomial_features),("lr",lr)])
    pipeline.fit(x_train, y_train)
    
    # 교차 검증으로 다항 회귀를 평가
    scores = cross_val_score(pipeline, x_train, y_train, scoring="neg_mean_squared_error", cv=5)
    coe = pipeline.named_steps['lr'].coef_
    print("Degree {0} 회귀 계수는 {1} ".format(degrees[i], np.round(coe,4)))
    print("MSE 는 : {0}".format(-1*np.mean(scores)))
    
   

KeyboardInterrupt: 

In [ ]:
 Regularized Linear Models - Ridge, Lasso
    Ridge Regression
    : L2 규제 로 w의 제곱 값에 규제

In [12]:
ridge = Ridge(alpha = 10)
# 교차 검증인데;;
neg_mse_score = cross_val_score(ridge, x_data, y_target, scoring="neg_mean_squared_error", cv=5)
rmse_score = np.sqrt(-1*neg_mse_score)
avg_rmse = np.mean(rmse_score)
print("5 folds의 개별 Negative MSE score:", np.round(neg_mse_score,3))
print("5 folds의 개별 RMSE score:", np.round(rmse_score,3))
print("5 folds의 평균 RMSE score:", np.round(avg_rmse, 3))

5 folds의 개별 Negative MSE score: [-19.166 -20.461 -19.436 -19.702 -99.237]
5 folds의 개별 RMSE score: [4.378 4.523 4.409 4.439 9.962]
5 folds의 평균 RMSE score: 5.542


In [13]:
alphas = [0, 0.1, 1, 10, 100, 120, 140]
for alpha in alphas:
    ridge = Ridge(alpha = alpha)
    # 교차 검증인데;;
    neg_mse_score = cross_val_score(ridge, x_data, y_target, scoring="neg_mean_squared_error", cv=5)
    rmse_score = np.sqrt(-1*neg_mse_score)
    avg_rmse = np.mean(rmse_score)
    print("alpha {0} 의 5 folds의 평균 RMSE score:".format(alpha), np.round(avg_rmse, 3))

alpha 0 의 5 folds의 평균 RMSE score: 5.542
alpha 0.1 의 5 folds의 평균 RMSE score: 5.542
alpha 1 의 5 folds의 평균 RMSE score: 5.542
alpha 10 의 5 folds의 평균 RMSE score: 5.542
alpha 100 의 5 folds의 평균 RMSE score: 5.545
alpha 120 의 5 folds의 평균 RMSE score: 5.546
alpha 140 의 5 folds의 평균 RMSE score: 5.547


In [ ]:
 라쏘 회귀

In [14]:
from sklearn.linear_model import Lasso, ElasticNet

In [15]:
# 곅
def get_linear_reg_eval (model_name, params=None, x_data_n=None, y_target_n = None, verbose=True):
    coeff_df = pd.DataFrame()
    print("####",model_name,"####")
    # alpha 값을 바꿔가며 반복
    for param in params:
        if model_name =='Ridge': model = Ridge(alpha=param)
        elif model_name =='Lasso': model = Lasso(alpha=param)
        elif model_name =='ElasticNet': model = ElasticNet(alpha=param, l1_ratio=0.7)
          
        neg_mse_scores = cross_val_score(model, x_data_n, y_target_n, scoring="neg_mean_squared_error", cv=5)
        avg_rmse = np.mean(np.sqrt(-1*neg_mse_scores))
        print("alpha {0} 의 5 folds의 평균 RMSE score:{1:.3f}".format(param, avg_rmse))
        # 모델 학습 시키기
        model.fit(x_data_n, y_target_n)
        coeff = pd.Series(data=model.coef_, index=x_data_n.columns)
        colname = 'alpha:'+str(param)
        coeff_df[colname] = coeff
    return coeff_df
          

In [16]:
lasso_alphas = [0.07, 0.1, 0.5,1,3]
coe_lasso_df = get_linear_reg_eval('Lasso', params=lasso_alphas, x_data_n = x_data, y_target_n=y_target)

#### Lasso ####
alpha 0.07 의 5 folds의 평균 RMSE score:5.945
alpha 0.1 의 5 folds의 평균 RMSE score:6.273
alpha 0.5 의 5 folds의 평균 RMSE score:10.186
alpha 1 의 5 folds의 평균 RMSE score:10.186
alpha 3 의 5 folds의 평균 RMSE score:10.186


In [ ]:
 엘라스틱넷 회귀

In [17]:
elastic_alphas = [0.07, 0.1, 0.5, 1, 3]
coeff_elastic_df = get_linear_reg_eval('ElasticNet', params=elastic_alphas, x_data_n = x_data, y_target_n=y_target)

#### ElasticNet ####


KeyboardInterrupt: 

In [ ]:
====================================================================================
여기까지 회귀의 다양한 방법
다음 노트에서 데이터 변환

In [19]:
from sklearn.preprocessing import StandardScaler , MinMaxScaler, PolynomialFeatures
# StandardScaler : 표준 정규분포
# MinMaxScaler : 최대값/최소값 정규화
# 로그 변환

In [20]:
# 기존 데이터 스케일 변환 시키는 함수
def get_scaled_data(method='None', p_degree=None, input_data=None):
    if method == 'Standard':
        scaled_data = StandardScaler().fit_transform(input_data)
    elif method == 'MinMax':
        scaled_data = MinMaxScaler().fit_transform(input_data)
    elif method == 'Log':
        scaled_data = np.log1p(input_data)
    else:
        scaled_data = input_data
        
    if p_degree != None:
        scaled_data = PolynomialFeatures(degree=p_degree, include_bias=False).fit_transform(scaled_data)
        
    return scaled_data

In [21]:
def get_linear_reg_eval (model_name, params=None, x_data_n=None, y_target_n = None, verbose=True):
    coeff_df = pd.DataFrame()
    print("####",model_name,"####")
    # alpha 값을 바꿔가며 반복
    for param in params:
        if model_name =='Ridge': model = Ridge(alpha=param)
        elif model_name =='Lasso': model = Lasso(alpha=param)
        elif model_name =='ElasticNet': model = ElasticNet(alpha=param, l1_ratio=0.7)
          
        neg_mse_scores = cross_val_score(model, x_data_n, y_target_n, scoring="neg_mean_squared_error", cv=5)
        avg_rmse = np.mean(np.sqrt(-1*neg_mse_scores))
        print("alpha {0} 의 5 folds의 평균 RMSE score:{1:.3f}".format(param, avg_rmse))
        # 모델 학습 시키기
        model.fit(x_data, y_target)                
        coeff = pd.Series(data=model.coef_, index=x_data.columns)
        colname = 'alpha:'+str(param)
        coeff_df[colname] = coeff
    return coeff_df
          

In [22]:
alphas = [0.1 , 1, 10, 100]
scale_methods = [(None, None),('Standard', None),('Standard', 2),('MinMax', None),('MinMax', 2),('Log', None)]

for scale in scale_methods:
    x_data_scaled = get_scaled_data(method=scale[0], p_degree=scale[1], input_data=x_data)    
    print("\n 변환유형 :{0}, Degree : {1}".format(scale[0], scale[1]))
    get_linear_reg_eval('Ridge', params=alphas, x_data_n = x_data_scaled, y_target_n=y_target, verbose=False)


 변환유형 :None, Degree : None
#### Ridge ####
alpha 0.1 의 5 folds의 평균 RMSE score:4.548
alpha 1 의 5 folds의 평균 RMSE score:4.548
alpha 10 의 5 folds의 평균 RMSE score:4.548
alpha 100 의 5 folds의 평균 RMSE score:4.551

 변환유형 :Standard, Degree : None
#### Ridge ####
alpha 0.1 의 5 folds의 평균 RMSE score:4.548
alpha 1 의 5 folds의 평균 RMSE score:4.548
alpha 10 의 5 folds의 평균 RMSE score:4.548
alpha 100 의 5 folds의 평균 RMSE score:4.548

 변환유형 :Standard, Degree : 2
#### Ridge ####


MemoryError: Unable to allocate 363. MiB for an array with shape (1080000, 44) and data type float64